In [2]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("detach", True)
ser = Service("C:\Program Files (x86)\chromedriver.exe")

DELAY = 2

# Code for getting Nursery List (Wildflower)


In [18]:
driver = webdriver.Chrome(service=ser, options=options)
driver.get("https://www.wildflower.org/suppliers/search.php?start=0&pagecount=100")

In [19]:
table = driver.find_element(By.CSS_SELECTOR,"div table")

In [42]:
html_list=  pd.read_html(table.get_attribute("outerHTML"))


In [43]:
df = html_list[0]
df.columns = df.iloc[0,:].to_list()

In [44]:
df.drop(index=df.index[0], axis=0, inplace=True)
df.drop(index=df.index[-1], axis=0, inplace=True)

In [45]:
df

,name,business category,associate,% native,location
1,A Touch O' Green Landscape & Gardens LLC,NurserySeed CompanyLandscape ProfessionalEnvir...,Yes,75,"Austin, TX"
2,'Ahakhav Tribal Preserve,Nursery,Yes,100,"Parker , AZ"
3,Agrivestment Ltd.,Seed Company,Yes,50,"Tacoma, WA"
4,Alexander Landscaping & Plant Farm,Nursery,Yes,50,"Davie, FL"
5,Alkane Nursery,Nursery,Yes,75,"Kapa|au, HI"
...,...,...,...,...,...
96,Florida Native Plants Nursery & Landscaping,NurseryLandscape ProfessionalEnvironmental Con...,Yes,75,"Sarasota, FL"
97,From Seeds to Home Nursery,Nursery,Yes,75,"San Angelo, TX"
98,Front Range Environmental,Landscape ProfessionalEnvironmental Consultant,Yes,75,"McHenry , IL"
99,Full Circle Gardens,Nursery,Yes,75,"Essex, VT"


# Cleaning Data

In [3]:
full_df = pd.read_csv("WildflowerNurseryList.csv",index_col=0)
full_df["State"] = full_df["location"].apply(lambda x: x.split(",")[1].strip() if isinstance(x,str) else x) #location is city,state
full_df["City"] = full_df["location"].apply(lambda x: x.split(",")[0] if isinstance(x,str) else x)
full_df.columns = pd.Series(full_df.columns).str.capitalize()
full_df.drop("Location",axis=1,inplace=True)


In [4]:
state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "HI", "ID", "IL", "IN", 
"IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", 
"NY", "NC", "ND", "OH", "OK", "OR", "PA", "PR", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "VI", 
"WA", "WV", "WI", "WY"]

In [5]:
import numpy as np
full_df["State"] = full_df["State"].apply(lambda x: x if x in state_codes else np.nan)

In [6]:
full_df.dropna(subset=["State"],inplace=True)
full_df.to_csv("WildflowerNurseryList2.csv")

# Code for scraping and cleaning ---

In [13]:
driver = webdriver.Chrome(service=ser, options=options)
driver.get("https://www.nurserytrees.com/States/state%20Alabama.htm")

In [22]:
tables = driver.find_elements(By.CSS_SELECTOR,"table")

full_table = None
for table in tables:
    section_df = pd.read_html(table.get_attribute("outerHTML"))[0]
    if full_table is None:
        full_table = section_df
    else:
        full_table = pd.concat([full_table,section_df])




In [11]:
html_list

[                0                              1  \
 0      ADAMSVILLE           LANDSCAPE SOURCE INC   
 1       ALABASTER           DOUG'S LAWN CARE INC   
 2       ALABASTER                  FAIRWAY LAWNS   
 3       ALABASTER             WALMART GARDEN CTR   
 4     ALBERTVILLE                A-1 MILL WRIGHT   
 5     ALBERTVILLE  ROBERTS LANDSCAPE & TURF MGMT   
 6  ALEXANDER CITY             BETTE'S GARDEN CTR   
 7  ALEXANDER CITY             WALMART GARDEN CTR   
 
                                2           3  
 0      1985 VETERANS MEMORIAL DR  2057919846  
 1                2562 TAHITI TER  2055316465  
 2      1510 SIMMSVILLE RD # 1800  2059782060  
 3    630 COLONIAL PROMENADE PKWY  2056200360  
 4           3124 AL HIGHWAY 75 N  2568918543  
 5  ROBERTS LANDSCAPE & TURF MGMT  2568917865  
 6             1217 TALLAPOOSA ST  2563291115  
 7               2643 HIGHWAY 280  2562340316  ]

In [5]:
df = html_list[0]
df.head()

,0,1,2,3
0,ADAMSVILLE,LANDSCAPE SOURCE INC,1985 VETERANS MEMORIAL DR,2057919846
1,ALABASTER,DOUG'S LAWN CARE INC,2562 TAHITI TER,2055316465
2,ALABASTER,FAIRWAY LAWNS,1510 SIMMSVILLE RD # 1800,2059782060
3,ALABASTER,WALMART GARDEN CTR,630 COLONIAL PROMENADE PKWY,2056200360
4,ALBERTVILLE,A-1 MILL WRIGHT,3124 AL HIGHWAY 75 N,2568918543


In [9]:
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", 
"Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", 
"Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
"New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolin","North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolin", "South Dakota", "Tennessee",
 "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

states = list(map(lambda x: "%20".join(x.split(" ")),states))


In [10]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New%20Hampshire',
 'New%20Jersey',
 'New%20Mexico',
 'New%20York',
 'North%20Carolin',
 'North%20Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode%20Island',
 'South%20Carolin',
 'South%20Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West%20Virginia',
 'Wisconsin',
 'Wyoming']